In [2]:
import os
import numpy as np
import cv2

import matplotlib.pyplot as plt
from keras.models import load_model
from tqdm  import tqdm
import glob
from datetime import datetime

import tensorflow as tf

# Add the upper level directory to the path
import sys
sys.path.append('..')
import imutils as im


# Overall approach for Cell Content Identification

The following approach is taken from DeepBee https://github.com/AvsThiago/DeepBee-source. The preprocessing was additionally created to remove/reduce the number of bees on the beehive frame and provide visibility of the uncovered cells. 

-foreground substraction
    some method to remove the bees covering the openings of the cells
    
-image segmentation
    
-classification
    capped honey cells
    capped brood cells
    pollen cells
    open honey cells
    open brood cells (empty)
    open brood cells(eggs)
    open brood cells(larvae)
    other

Since the code of the DeepBee project is not freely accessible, the compiled .exe program was tested to classify the cell content.


The images taken for testing the code can be found in the folder 24.04_observation_OH. Computationally removing the bees using the provided photos was done using different number of images. Since the bee cluster/colony is mostly located at the same area of the frame, a dark shade at that region can be seen. The cells themselves are still not very well visible for the tested algorithms. 

To remove the bees, the two functions createBackgroundSubtractorMOG2 and createBackgroundSubtractorKNN were used with different parameters. The chosen functions seem to be computing the desired output but based on the input images provided, a complete removal of the honey bees on the frame seems to be rather complicated. For all parameters and sufficiently large number of images (some 100), bees themselves vanish but a dark region covering the cells appears. -> hard to extract data in region of bee cluster

backSub = cv2.createBackgroundSubtractorMOG2(history=len(images), varThreshold=16, detectShadows=False)
backSub = cv2.createBackgroundSubtractorKNN(detectShadows=True)

In [3]:
def get_all_file_paths(directory):
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

def load_images_from_folder(image_paths):
    images = []
    for filename in image_paths:  # Adjust the extension if needed
        img = cv2.imread(filename)
        if img is not None:
            images.append(img)
    return images

def remove_moving_objects(images):

    # Create background subtractor
#     backSub = cv2.createBackgroundSubtractorMOG2(history=len(images), varThreshold=16, detectShadows=False)
    backSub = cv2.createBackgroundSubtractorKNN(detectShadows=True) # 2 different methods to subtract the backgrounds

    # Apply the background subtractor to each image to build the background model
    for img in images:
        backSub.apply(img)

    # Now, obtain the background model
    background_image = backSub.getBackgroundImage()

    return background_image

# Preprocessing

In [6]:
# Example usage
root = '/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09_observation_OH/Images/'

directory = root + r"\24.04_observation_OH\24.04_observation_OH\Images\h2r2_1minute\all_images".format(drive_letter)

for num_days in [0.5,1, 2, 3]:
    first_index = int(num_days*60*24) # last n hours
    downsampling = 20 #60

    image_paths_all = get_all_file_paths(directory)
    image_paths_most_recent = image_paths_all[-first_index:]
    image_paths = image_paths_most_recent[::downsampling]

    images = load_images_from_folder(image_paths)

    now = datetime.now()

    # Format it in a human-readable way
    timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

    # Create a directory to save the processed images
    output_folder = root +"\output\{}_imgs{}_numdays{}_step{}".format(timestamp, len(images), num_days, downsampling)
    os.makedirs(output_folder, exist_ok=True)

    # Example usage

    if len(images) > 0:
        print("Number of images:", len(images))
        result_image = remove_moving_objects(images)

        background_filename = output_folder + '\\background_image{}_imgs{}_numdays{}_step{}.jpg'.format(first_index, len(images), num_days, downsampling)
        print(background_filename)
        cv2.imwrite(background_filename, result_image)
    #     cv2.imshow('Result', result_image)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()
    else:
        print("No images found in the specified folder.")
    del images, result_image
# reads the last background image
result_image = cv2.imread(background_filename)


Number of images: 36
E:\MobotsGroup\output\2024-09-11_10-23-03_imgs36_numdays0.5_step20\background_image720_imgs36_numdays0.5_step20.jpg
Number of images: 72
E:\MobotsGroup\output\2024-09-11_10-23-59_imgs72_numdays1_step20\background_image1440_imgs72_numdays1_step20.jpg
Number of images: 144


KeyboardInterrupt: 

In [5]:

background_filename = r"E:\MobotsGroup\output\2024-08-09_15-33-50_imgs216_numdays3_step20\background_image4320_imgs216_numdays3_step20.jpg"
result_image = cv2.imread(background_filename)
print(background_filename)



E:\MobotsGroup\output\2024-08-09_15-33-50_imgs216_numdays3_step20\background_image4320_imgs216_numdays3_step20.jpg


In [ ]:
# additional work done to create filter indicating the location of bees:

In [14]:
def compare_images(img1, img2, scale=1.0):
    """
    Compare two images and create a new image highlighting the changes.
    
    Parameters:
    - img1: First image (cv2 image object)
    - img2: Second image (cv2 image object)
    - scale: Scaling factor for the differences
    
    Returns:
    - diff_img: Image representing the pixel-wise differences
    """
    if img1.shape != img2.shape:
        raise ValueError("Input images must have the same dimensions")

    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)

    diff_img = np.abs(img1 - img2)

    # Scale the difference image to make the changes more visible
    diff_img *= scale

    # Clip values to the valid range [0, 255] and convert to uint8
    diff_img = np.clip(diff_img, 0, 255).astype(np.uint8)

    return diff_img

def brighten_image_with_mask(main_img, mask_img, max_increase=1.0):
    """
    Brighten the main image using the mask image, proportional to the mask's pixel values.
    
    Parameters:
    - main_img: Main image (cv2 image object, should be uint8)
    - mask_img: Mask image (cv2 image object, should be uint8, same size as main_img)
    - max_increase: Maximum scaling factor to increase brightness based on mask

    Returns:
    - brightened_img: Image with brightness increased based on mask
    """
    if main_img.shape != mask_img.shape:
        raise ValueError("Main image and mask image must have the same dimensions")

    main_img = main_img.astype(np.float32)

    # Normalize mask to range [0, 1] by dividing by 255 (if the mask is uint8)
    mask_img = mask_img.astype(np.float32) / 255.0

    # Increase brightness proportional to the mask values
    brightened_img = main_img + (mask_img * max_increase * 255)

    brightened_img = np.clip(brightened_img, 0, 255).astype(np.uint8)
    
    print("Min pixel value:", mask_img.min())
    print("Max pixel value:", mask_img.max())
    mask_img = (mask_img.astype(np.float32) * 255.0).astype(np.uint8)

    return brightened_img, mask_img

def scale_image(image, scale_factor = 0.5):
    height, width = image.shape[:2]

    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)

    # Resize the image while keeping aspect ratio
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_image


# recursive call of the functions to remove the bees:
def recursive_bee_remover(first_image, second_image, max_increase = 1.0):
    diff_img = compare_images(first_image, second_image, scale=1.0)
    brightened_img, img_mask = brighten_image_with_mask(first_image, diff_img, max_increase)
    return brightened_img, img_mask

def mask_average(mask_list):
    average = np.zeros_like(mask_list[0], dtype=np.float32)  # Use float32 for precision
    for mask in mask_list:
        mask = mask.astype(np.float32)  # Convert each mask to float32 to avoid overflow issues
        average += mask 
    average = average / len(mask_list)
    
    average = np.clip(average, 0, 255).astype(np.uint8)
    
    return average
    


def store_image(image, root, folder_name, name_extension):
    folder_path = root + "\\" + folder_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'Folder created at: {folder_path}')
    else:
        print(f'Folder already exists at: {folder_path}', "now saving", name_extension)    
    file_path =  str(root + folder_name + '\\' + folder_name + name_extension+".png")
    print(file_path)
    cv2.imwrite(file_path, image)
    

In [18]:
# get files in base directory
"""
base_dir: directory containing the set of images of the hive to remove the bees
root : root directory where the computed masks and resulting images will be stored at
"""

# base directory for images (input)
base_dir = r"C:\Users\raf-k\Desktop\MOBOTS Group project\cell classification\code\images"
base_dir = r"G:\MobotsGroup\24.04_observation_OH\24.04_observation_OH\Images\h2r3_1minute\all_images"
# directory for output 
root = r"G:\MobotsGroup\cell_classification_KNN"
background_filenames = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if os.path.isfile(os.path.join(base_dir, f))]


In [11]:
root

'E:\\MobotsGroup\\cell_classification_KNN'

In [55]:
first_image = cv2.imread(background_filenames[0])
# background_filenames = background_filenames[::3]
image_list = []
mask_list = []
num_images = 40
algorithm_name = "\\recursive_{}_imgs".format(str(num_images))
for filename in background_filenames[1:num_images]:
    second_image = cv2.imread(filename)
    result_img, img_mask = recursive_bee_remover(first_image, second_image, 1)
    print(background_filenames.index(filename)/len(background_filenames)*100, "%")
    print(root + algorithm_name)

    store_image(img_mask, root + algorithm_name, "\\recursive_masks_{}".format(num_images), "_mask_recursive_{}".format(background_filenames.index(filename)))
    store_image(result_img, root + algorithm_name, "\\recursive_subimages_{}".format(num_images), "_subimages_recursive_{}".format(background_filenames.index(filename)))

    image_list.append(result_img)
    mask_list.append(img_mask)
    first_image = second_image
        
result = remove_moving_objects(image_list)
store_image(result, root + algorithm_name,'', "result_recursive_{}".format(num_images))
resized_image = scale_image(result, 0.5)
# cv2.imshow('Brightened Img recursive n to n+1', resized_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Min pixel value: 0.0
Max pixel value: 0.8627451
1.1494252873563218 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_1
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_masks_40_mask_recursive_1.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_subimages_40 now saving _subimages_recursive_1
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_subimages_40\\recursive_subimages_40_subimages_recursive_1.png
Min pixel value: 0.0
Max pixel value: 0.8627451
2.2988505747126435 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_2
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_masks_40_

Min pixel value: 0.0
Max pixel value: 0.84705883
16.091954022988507 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_14
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_masks_40_mask_recursive_14.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_subimages_40 now saving _subimages_recursive_14
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_subimages_40\\recursive_subimages_40_subimages_recursive_14.png
Min pixel value: 0.0
Max pixel value: 0.8980392
17.24137931034483 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_15
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_mask

Min pixel value: 0.0
Max pixel value: 0.85882354
31.03448275862069 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_27
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_masks_40_mask_recursive_27.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_subimages_40 now saving _subimages_recursive_27
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_subimages_40\\recursive_subimages_40_subimages_recursive_27.png
Min pixel value: 0.0
Max pixel value: 0.7921569
32.18390804597701 %
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\\recursive_masks_40 now saving _mask_recursive_28
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\recursive_masks_40\\recursive_masks

Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\ now saving result_recursive_40
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\result_recursive_40.png


In [56]:
####  using the average of the consecutive masks
first_image = cv2.imread(background_filenames[0])
store_image(first_image, root + algorithm_name, '', "bees_averaged_original_{}".format(num_images))

image_list = []
mask_list = []

num_images = 40
skipped = 5 # to increase time between images
algorithm_name = "\\average_{}_imgs_skipped{}".format(num_images,skipped)
for filename in background_filenames[1:skipped * num_images][::skipped]:
    second_image = cv2.imread(filename)
    result_img, img_mask = recursive_bee_remover(first_image, second_image, 1.0)# here used to compute the mask
    print(filename)
    print(background_filenames.index(filename)/len(background_filenames) * 100, "%")
    image_list.append(result_img)
    mask_list.append(img_mask)
    store_image(img_mask, root + algorithm_name, "\\average_masks_{}".format(num_images), "_mask_averaged_{}".format(background_filenames.index(filename)))

    first_image = second_image
    
average_mask = mask_average(mask_list)
store_image(average_mask, root + algorithm_name,'', "mask_averaged_{}".format(num_images))

bees_removed_image = remove_moving_objects(image_list)
store_image(bees_removed_image, root + algorithm_name, '', "bees_removed_averaged_{}".format(num_images))


for max_increase in [.2, .4, .6, .8, 1, 1.,2, 1.4, 10]:
    # added gaussian blur over average mask 
    result, img_maskunused = brighten_image_with_mask(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_averaged_{}_".format(num_images)+str(max_increase))

    average_mask = cv2.GaussianBlur(average_mask, (7,7), 0)
    store_image(average_mask, root + algorithm_name, '', "average_mask_blurr{}_".format(num_images)+str(max_increase))

    result, img_maskunused = brighten_image_with_mask(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_averaged_blurr{}_".format(num_images)+str(max_increase))

    
# for filename in background_filenames:
#     image_list.append(cv2.imread(filename))
    
resized_image = scale_image(result, 0.5)
# cv2.imshow('Brightened Img with averaged mask', resized_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Folder already exists at: E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\ now saving bees_averaged_original_40
E:\MobotsGroup\cell_classification_KNN\recursive_40_imgs\bees_averaged_original_40.png
Min pixel value: 0.0
Max pixel value: 0.8627451
E:\MobotsGroup\24.04_observation_OH\24.04_observation_OH\Images\h2r3_1minute\all_images\hive2_rpi3_240419-184802Z.jpg
1.1494252873563218 %
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\\average_masks_40 now saving _mask_averaged_1
E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\average_masks_40\\average_masks_40_mask_averaged_1.png
Min pixel value: 0.0
Max pixel value: 0.9137255
E:\MobotsGroup\24.04_observation_OH\24.04_observation_OH\Images\h2r3_1minute\all_images\hive2_rpi3_240420-072102Z.jpg
6.896551724137931 %
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\\average_masks_40 now saving _mask_averaged_6
E:\MobotsGroup\cell_classifi

Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\ now saving bees_removed_averaged_40
E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\bees_removed_averaged_40.png
Min pixel value: 0.003921569
Max pixel value: 0.3254902
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\ now saving bees_removed_brightened_averaged_40_0.2
E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\bees_removed_brightened_averaged_40_0.2.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\ now saving average_mask_blurr40_0.2
E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\average_mask_blurr40_0.2.png
Min pixel value: 0.007843138
Max pixel value: 0.2627451
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\average_40_imgs_skipped5\ now saving bees_removed_brightened_averaged_blurr40_0.2
E:\MobotsGroup\cell_classification_KNN\average_40_i

In [19]:
# # using optical flow to create a filter
def recursive_bee_remover2(first_image, second_image, max_increase = 1.0):
    """ bee remover using the optical flow instead of difference of two consecutive images to compute the mask"""
    flow, mag_of, of_ang = im.compute_dense_optical_flow(first_image, second_image)
    brightened_img, img_mask = brighten_image_with_mask(first_image, mag_of, max_increase)
    return brightened_img, img_mask  

num_images = 40
skipped = 5 # to increase time between images
algorithm_name = "\\optical_flow_{}_imgs_skipped{}".format(num_images, skipped)


####  using the optical flow of the consecutive masks
first_image = cv2.imread(background_filenames[0], cv2.IMREAD_GRAYSCALE)
store_image(first_image, root + algorithm_name, '', "bees_optical_flow_original_{}".format(num_images))

image_list = []
mask_list = []

for filename in background_filenames[1:skipped * num_images][::skipped]:
    second_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    
    result_img, img_mask = recursive_bee_remover2(first_image, second_image, 1.0) # here used to compute the mask
#     print(filename)
#     print(background_filenames.index(filename)/len(background_filenames) * 100, "%")
    image_list.append(result_img)
    img_mask = img_mask*10 # scale pixel values to obtain a higher value
    mask_list.append(img_mask)
    store_image(img_mask, root + algorithm_name, "\\optical_flow_masks_{}".format(num_images), "_mask_optical_flow_{}".format(background_filenames.index(filename)))

    first_image = second_image
    
    
average_mask = mask_average(mask_list)
if True:
    average_mask = cv2.GaussianBlur(average_mask, (7,7), 0)

store_image(average_mask, root + algorithm_name, '', "mask_optical_flow_{}".format(num_images))

bees_removed_image = remove_moving_objects(image_list)
store_image(bees_removed_image, root + algorithm_name, '', "bees_removed_optical_flow_{}".format(num_images))


for max_increase in [1, 10, 20, 30, 40]:# [0.1, 1, 2, 3, 4]:#
    # added gaussian blur over average mask 
    result, img_maskunused = brighten_image_with_mask(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_optical_flow_{}_".format(num_images)+str(max_increase))

    average_mask = cv2.GaussianBlur(average_mask, (7,7), 0)
    store_image(average_mask, root + algorithm_name, '', "optical_flow_mask_blurr{}_".format(num_images)+str(max_increase))

    result, img_maskunused = brighten_image_with_mask(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_optical_flow_blurr{}_".format(num_images)+str(max_increase))

    
# # for filename in background_filenames:
# #     image_list.append(cv2.imread(filename))
    
# resized_image = scale_image(result, 0.5)
# # cv2.imshow('Brightened Img with averaged mask', resized_image)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\ now saving bees_optical_flow_original_40
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\bees_optical_flow_original_40.png
Min pixel value: 1.6529096e-07
Max pixel value: 0.1248244
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\\optical_flow_masks_40 now saving _mask_optical_flow_1
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\optical_flow_masks_40\\optical_flow_masks_40_mask_optical_flow_1.png
Min pixel value: 2.6775103e-07
Max pixel value: 0.18217447
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\\optical_flow_masks_40 now saving _mask_optical_flow_6
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\optical_flow_masks_40\\optical_flow_masks_40_mask_optical_flow_6.png
Min pixel value: 8.16676e-08
Max pixel value: 0.2488744
Folder alre

Min pixel value: 0.0
Max pixel value: 0.47843137
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\ now saving bees_removed_brightened_optical_flow_blurr40_10
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\bees_removed_brightened_optical_flow_blurr40_10.png
Min pixel value: 0.0
Max pixel value: 0.47843137
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\ now saving bees_removed_brightened_optical_flow_40_20
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\bees_removed_brightened_optical_flow_40_20.png
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\ now saving optical_flow_mask_blurr40_20
G:\MobotsGroup\cell_classification_KNN\optical_flow_40_imgs_skipped5\optical_flow_mask_blurr40_20.png
Min pixel value: 0.0
Max pixel value: 0.47843137
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_

In [20]:
# # using optical flow to create a filter
def recursive_bee_remover2(first_image, second_image, max_increase = 1.0):
    """ bee remover using the optical flow instead of difference of two consecutive images to compute the mask"""
    flow, mag_of, of_ang = im.compute_dense_optical_flow(first_image, second_image)
    brightened_img, img_mask = brighten_image_with_mask_multiplication(first_image, mag_of, max_increase)
    return brightened_img, img_mask


def brighten_image_with_mask_multiplication(main_img, mask_img, max_increase=1.0):
    """
    Brighten the main image using the mask image, proportional to the mask's pixel values.
    
    Parameters:
    - main_img: Main image (cv2 image object, should be uint8)
    - mask_img: Mask image (cv2 image object, should be uint8, same size as main_img)
    - max_increase: Maximum scaling factor to increase brightness based on mask

    Returns:
    - brightened_img: Image with brightness increased based on mask
    """
    if main_img.shape != mask_img.shape:
        raise ValueError("Main image and mask image must have the same dimensions")

    main_img = main_img.astype(np.float32)

    # Normalize mask to range [0, 1] by dividing by 255 (if the mask is uint8)
    mask_img = mask_img.astype(np.float32) / 255.0
    
    if True:
        average_mask = cv2.GaussianBlur(mask_img, (21,21), 0)
    
    # Increase brightness proportional to the mask values
    brightened_img = cv2.multiply(main_img, mask_img, scale=1/255.0) * max_increase

    brightened_img = np.clip(brightened_img, 0, 255).astype(np.uint8)
    
    print("Min pixel value:", mask_img.min())
    print("Max pixel value:", mask_img.max())
    mask_img = (mask_img.astype(np.float32) * 255.0).astype(np.uint8)
    
    return brightened_img, mask_img

    

num_images = 40
skipped = 5 # to increase time between images
algorithm_name = "\\optical_flow_multiplication_{}_imgs_skipped{}".format(num_images, skipped)


####  using the optical flow of the consecutive masks
first_image = cv2.imread(background_filenames[0], cv2.IMREAD_GRAYSCALE)
store_image(first_image, root + algorithm_name, '', "bees_optical_flow_original_{}".format(num_images))

image_list = []
mask_list = []

for filename in background_filenames[1:skipped * num_images][::skipped]:
    second_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    
    result_img, img_mask = recursive_bee_remover2(first_image, second_image, 1.0) # here used to compute the mask
#     print(filename)
#     print(background_filenames.index(filename)/len(background_filenames) * 100, "%")
    image_list.append(result_img)
    img_mask = img_mask*10 # scale pixel values to obtain a higher value
    mask_list.append(img_mask)
    store_image(img_mask, root + algorithm_name, "\\optical_flow_masks_{}".format(num_images), "_mask_optical_flow_{}".format(background_filenames.index(filename)))

    first_image = second_image
    
    
average_mask = mask_average(mask_list)


store_image(average_mask, root + algorithm_name, '', "mask_optical_flow_{}".format(num_images))

bees_removed_image = remove_moving_objects(image_list)
store_image(bees_removed_image, root + algorithm_name, '', "bees_removed_optical_flow_{}".format(num_images))


for max_increase in [1, 10, 20, 30, 40]:# [0.1, 1, 2, 3, 4]:#
    # added gaussian blur over average mask 
    result, img_maskunused = brighten_image_with_mask_multiplication(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_optical_flow_{}_".format(num_images)+str(max_increase))

    average_mask = cv2.GaussianBlur(average_mask, (21,21), 0)
    store_image(average_mask, root + algorithm_name, '', "optical_flow_mask_blurr{}_".format(num_images)+str(max_increase))

    result, img_maskunused = brighten_image_with_mask_multiplication(bees_removed_image, average_mask, max_increase)
    store_image(result, root + algorithm_name, '', "bees_removed_brightened_optical_flow_blurr{}_".format(num_images)+str(max_increase))

    
# # for filename in background_filenames:
# #     image_list.append(cv2.imread(filename))
    
# resized_image = scale_image(result, 0.5)
# # cv2.imshow('Brightened Img with averaged mask', resized_image)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\ now saving bees_optical_flow_original_40
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\bees_optical_flow_original_40.png
Min pixel value: 1.6529096e-07
Max pixel value: 0.1248244
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\\optical_flow_masks_40 now saving _mask_optical_flow_1
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\optical_flow_masks_40\\optical_flow_masks_40_mask_optical_flow_1.png
Min pixel value: 2.6775103e-07
Max pixel value: 0.18217447
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\\optical_flow_masks_40 now saving _mask_optical_flow_6
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\optical_flow_masks_40\\optical_flow_masks_40_m

Min pixel value: 0.0
Max pixel value: 0.47058824
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\ now saving bees_removed_brightened_optical_flow_blurr40_1
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\bees_removed_brightened_optical_flow_blurr40_1.png
Min pixel value: 0.0
Max pixel value: 0.47058824
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\ now saving bees_removed_brightened_optical_flow_40_10
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\bees_removed_brightened_optical_flow_40_10.png
Folder already exists at: G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\ now saving optical_flow_mask_blurr40_10
G:\MobotsGroup\cell_classification_KNN\optical_flow_multiplication_40_imgs_skipped5\optical_flow_mask_blurr40_10.png
Min pixel value: 0.0
Max pixel val

# Observations

multiplication is overall noisier than addition, as it seems. 
when using multiplication, without gaussian blurr, a very noisy image is obtained with kernel size 7x7
first approaches with knn and mog2 averaging seem to remove the bees more reliably, yet, the dark shadows then created will have to be removed by detecting the darkest regions of the image and increase brightness in those regions

Folder already exists at: E:\MobotsGroup\cell_classification_KNN\\opticalFlow now saving of_mag
E:\MobotsGroup\cell_classification_KNN\opticalFlow\\opticalFlowof_mag.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\\opticalFlow now saving firstimag
E:\MobotsGroup\cell_classification_KNN\opticalFlow\\opticalFlowfirstimag.png
Folder already exists at: E:\MobotsGroup\cell_classification_KNN\\opticalFlow now saving secondimag
E:\MobotsGroup\cell_classification_KNN\opticalFlow\\opticalFlowsecondimag.png


# Implementation of approach of DeepBee project
All code hereafter is taken from or inspired by the DeepBee project. Applying it to the results did not seem to lead to satisfying results and the source code for changing the model itself could not be found. -> Might be worth creating a new model based on the image data of images gathered at Bassenges. The models provided by DeepBee did not provide satisfying results, not even after retraining with our data.

In [ ]:
# # Get a list of image paths
# # image_paths = sorted(glob(os.path.join(image_folder, '*.jpg')))  # Change the extension if necessary
# # image_paths

# # Initialize the background subtractor
# backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

# for i, image_path in enumerate(image_paths):
#     # Read the image
#     frame = cv2.imread(image_path)
    
#     # Apply the background subtractor
#     fg_mask = backSub.apply(frame)
    
#     # Optional: Improve mask using morphological operations
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
#     fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    
#     # Invert the mask to get the background (cells) only
#     bg_mask = cv2.bitwise_not(fg_mask)
    
#     # Use the mask to extract the background
#     bg = cv2.bitwise_and(frame, frame, mask=bg_mask)
    
#     # Save the result
#     output_path = os.path.join(output_folder, f'processed_{i:04d}.jpg')
#     cv2.imwrite(output_path, bg)

#     # Display the result for verification
#     cv2.imshow('Background', bg)
#     cv2.waitKey(30)  # Display each image for 30 ms

# cv2.destroyAllWindows()

In [ ]:
# # Load all images and store them in a list
# images = []
# for image_path in image_paths:
#     img = cv2.imread(image_path)
#     if img is not None:
#         images.append(img)

# # Convert the list of images to a NumPy array
# images_np = np.array(images, dtype=np.float32)

# # Compute the median along the time axis
# median_background = np.median(images_np, axis=0).astype(np.uint8)

# # Initialize the background subtractor
# backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)


# # Process each image by subtracting the median background
# for i, image_path in enumerate(image_paths):
#     # Read the current image
#     frame = cv2.imread(image_path)
    
#     # Subtract the median background
#     fg_mask = cv2.absdiff(frame, median_background)
    
#     # Convert to grayscale
#     fg_mask_gray = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2GRAY)
    
#     # Apply a binary threshold to get a binary mask
#     _, fg_mask_bin = cv2.threshold(fg_mask_gray, 30, 255, cv2.THRESH_BINARY)
    
#     # Optional: Improve mask using morphological operations
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     fg_mask_bin = cv2.morphologyEx(fg_mask_bin, cv2.MORPH_CLOSE, kernel)
#     fg_mask_bin = cv2.morphologyEx(fg_mask_bin, cv2.MORPH_OPEN, kernel)
    
#     # Invert the mask to get the background (cells) only
#     bg_mask = cv2.bitwise_not(fg_mask_bin)
    
#     # Use the mask to extract the background
#     bg = cv2.bitwise_and(frame, frame, mask=bg_mask)
    
#     # Save the result
#     output_path = os.path.join(output_folder, f'processed_{i:04d}.jpg')
#     cv2.imwrite(output_path, bg)

#     # Display the result for verification
#     cv2.imshow('Background', bg)
#     cv2.waitKey(30)  # Display each image for 30 ms

# cv2.destroyAllWindows()

# Enhance image

In [ ]:
def increase_contrast(image):
    # Convert to YUV color space
    yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    
    # Apply CLAHE to the Y channel (luminance)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    yuv[:, :, 0] = clahe.apply(yuv[:, :, 0])
    
    # Convert back to BGR color space
    enhanced_image = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    
    return enhanced_image

def sharpen_image(image):
    # Create a sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 4,-1],
                       [0, -1, 0]])
    
    # Apply the sharpening kernel to the image
    sharpened_image = cv2.filter2D(image, -1, kernel)
    
    return sharpened_image

def remove_distortion(image):    # TODO add if needed
    undistorted_image = image
    return undistorted_image
    


# Example usage
# Assuming result_image is the output from your background subtraction process
undistorted_image = remove_distortion(result_image)
contrast_image = increase_contrast(undistorted_image)
sharpened_image = sharpen_image(contrast_image)


# Save or display the result image


preprocessed_filename = output_folder + '\\output_image_contrast_sharpened.jpg'.format(first_index, downsampling)
cv2.imwrite(preprocessed_filename, result_image)
print("processed image successfully written to: ")
print(preprocessed_filename)
# cv2.imshow('Sharpened Image', sharpened_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Image segmentation

In [ ]:
processed_filename = r"E:\MobotsGroup\output\2024-08-12_09-21-01_imgs216_numdays3_step20\output_image_contrast_sharpened.jpg"
processed_img = cv2.imread(processed_filename)

print(processed_img.shape)
plt.imshow(processed_img)
plt.show()


In [ ]:

# # Image loading and resizing
# original_shape = (4608, 2592)  # Desired image shape
# img = processed_img


# def convert_bgr_to_rgb(img):
#     return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# # Image loading

# print(f"Original Shape: {img.shape[:2]}")

# # Resize to the desired shape if not already
# if img.shape[:2] != original_shape:
#     img = cv2.resize(img, (original_shape[0], original_shape[1]))  # (width, height)
# print(f"Resized Shape: {img.shape[:2]}")

# # Add mirrored border
# top_border = bottom_border = 184  # Adjust these based on desired overlap and slice size
# left_border = right_border = 148
# reflect = cv2.copyMakeBorder(img, top_border, bottom_border, left_border, right_border, cv2.BORDER_REFLECT)

# plt.imshow(convert_bgr_to_rgb(reflect))
# plt.title('Image with Mirrored Border')
# plt.axis('off')  # Hide axes
# plt.show()

# # Define overlap and step size
# overlap = 128
# slice_size = 512
# step_size = slice_size - overlap

# # Calculate positions for slicing
# pos_x = np.arange(0, original_shape[1] + 2 * left_border - slice_size + 1, step_size)
# pos_y = np.arange(0, original_shape[0] + 2 * top_border - slice_size + 1, step_size)

# # Generate slices
# slices = [np.s_[y[0]:y[1], x[0]:x[1]] for x in zip(pos_x, pos_x + slice_size) for y in zip(pos_y, pos_y + slice_size)]
# print(f"Examples of slices: {slices[:2]}")

# # Extract all the tiles and resize them to 128x128px (model input shape)
# IMG_WIDTH = 128
# IMG_HEIGHT = 128
# IMG_CHANNELS = 3

# X = np.zeros((len(slices), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

# for j, sl in enumerate(slices):
#     print(j, sl)
#     if reflect[sl].size == 0:
#         print(f"Skipping empty slice at index {j}")
#         continue
#     else:# Corrected dimensions for cv2.resize: (width, height)
#         X[j] = cv2.resize(reflect[sl], (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)

# # Plotting the tiles extracted
# def plot_tiles(X, func) -> None:
#     num_tiles_y = len(pos_y)
#     num_tiles_x = len(pos_x)

#     f, axarr = plt.subplots(num_tiles_y, num_tiles_x, figsize=(10, 10))
#     f.subplots_adjust(hspace=-0.8, wspace=.1)
#     img_index = 0

#     for i in range(num_tiles_y):
#         for j in range(num_tiles_x):
#             if img_index < len(X):  # Check to avoid index out of bounds
#                 axarr[i, j].imshow(func(X[img_index]), vmin=0, vmax=255)
#             axarr[i, j].axis('off')
#             img_index += 1

#     plt.show()

# plot_tiles(X, convert_bgr_to_rgb)

In [ ]:
def convert_bgr_to_rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
original_shape = processed_img.shape[:2]
print(f"Original Shape: {original_shape}", "ratio 16:9 ?")



if original_shape != (2592, 4608):
    resized_img = cv2.resize(processed_img, (2592, 4608))

    resized_shape = resized_img.shape[:2]
    print(f"Resized image, shape now: {resized_shape}")
    plt.imshow(resized_img)
    plt.show()
else:
    resized_img = processed_img


In [ ]:
# Adds a frame consisting of the mirrored border of the image
reflect = cv2.copyMakeBorder(resized_img, 184, 184, 148, 148, cv2.BORDER_REFLECT)
reflect.shape

In [ ]:
plt.imshow(convert_bgr_to_rgb(reflect))
plt.show()

In [ ]:
# overlap = 64
# slice_size = 512
# step_size = slice_size - overlap  
# reflect_x = 2960
# reflect_y = 4904

# pos_x = np.arange(0, reflect_x - slice_size + 1, step_size)
# pos_y = np.arange(0, reflect_y - slice_size + 1, step_size)

# slices = [np.s_[ x[0]:x[1], y[0]:y[1]] for x in zip(pos_x, pos_x + slice_size) for y in zip(pos_y, pos_y + slice_size)  ]
# print(f"Exemples of slices: {slices[:2]}")

In [ ]:
# IMG_WIDTH = 128
# IMG_HEIGHT = 128
# IMG_CHANNELS = 3

# X = np.zeros((len(slices), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

# good_indices = 0    
# for j, sl in enumerate(slices): 
    
#     print(j, sl, good_indices)
#     slice_img = reflect[sl]
#     print(f"Processing slice {j}, Slice shape: {slice_img.shape}")

# #     Check if the slice is empty or has unexpected dimensions
#     if slice_img.size == 0:
#         print(f"--- ----  Skipping empty slice at index {j}")
#         continue
#     elif slice_img.shape[0] != 512 or slice_img.shape[1] != 512:
#         print(f"Skipping invalid slice with shape {slice_img.shape} at index {j}")
#         continue
#     else:
#         X[good_indices] = cv2.resize(reflect[sl],(IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
#         good_indices +=1

In [ ]:
# for i, slice_img in enumerate(X):
#     if slice_img.sum() == 0:  # This checks if the slice is completely black
#         print(f"Slice {i} is empty or black")
# print(len(X))

In [ ]:
def cut_tiles(image):
    overlap = 64
    slice_size = 512
    step_size = slice_size - overlap  
    reflect_x = 2960
    reflect_y = 4904

    pos_x = np.arange(0, reflect_x - slice_size + 1, step_size)
    pos_y = np.arange(0, reflect_y - slice_size + 1, step_size)

    slices = [np.s_[ x[0]:x[1], y[0]:y[1]] for x in zip(pos_x, pos_x + slice_size) for y in zip(pos_y, pos_y + slice_size)  ]
    print(f"Exemples of slices: {slices[:2]}")
    
    IMG_WIDTH = 128
    IMG_HEIGHT = 128
    IMG_CHANNELS = 3

    X = np.zeros((len(slices), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

    good_indices = 0    
    for j, sl in enumerate(slices): 

        slice_img = image[sl]
#         print(f"Processing slice {j}, Slice shape: {slice_img.shape}")

    #     Check if the slice is empty or has unexpected dimensions
        if slice_img.size == 0:
            print(f"--- ----  Skipping empty slice at index {j}")
            continue
        elif slice_img.shape[0] != 512 or slice_img.shape[1] != 512:
            print(f"Skipping invalid slice with shape {slice_img.shape} at index {j}")
            continue
        else:
            X[good_indices] = cv2.resize(image[sl],(IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
            good_indices +=1
    return X
    
X = cut_tiles(reflect)

In [ ]:
# Plotting the tiles extracted
def plot_tiles(X, func) -> None:
    f, axarr = plt.subplots(6, 11,  figsize=(10,10))
    f.subplots_adjust(hspace=-0.8, wspace=.1)
    img_index = 0
    

    for j in range(6):
        for i in range(10):
            while X[img_index].sum() == 0:  # This checks if the slice is completely black
                print(f"Slice {img_index} is empty or black")
                img_index += 1
            axarr[j,i].imshow(func(X[img_index]), vmin=0, vmax=255)
            axarr[j,i].axis('off')
            img_index += 1
            
plot_tiles(X, convert_bgr_to_rgb)


In [ ]:
# print(len(np.arange(0, 4608 - 512 + 1, 384)),   len(np.arange(0, 2592 - 512 + 1, 384)))

In [ ]:
model_path = root + r"\Models\segmentation.h5"
model_path = r"E:\MobotsGroup\Models\segmentation.h5"
print(model_path)
model = load_model(model_path)

In [ ]:
print(tf.__version__)

# Detect hexagonal structures on each slice

In [ ]:
def detect_hexagons(image, draw_hexagons=True):
    # Load the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Use Canny Edge Detector
    edges = cv2.Canny(blurred, 50, 150)
    
    # Detect contours in the image
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    hexagon_centers = []
    
    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Check if the polygon has 6 sides (i.e., a hexagon)
        if len(approx) == 6:
            # Compute the center of the hexagon
            M = cv2.moments(approx)
            if M["m00"] != 0:
                center_x = int(M["m10"] / M["m00"])
                center_y = int(M["m01"] / M["m00"])
            else:
                continue
            
            hexagon_centers.append((center_x, center_y))
            
            if draw_hexagons:
                # Draw the hexagon and its center on the image
                cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
                cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
    
    # Display the result if draw_hexagons is True
    if draw_hexagons:
        cv2.imshow('Hexagons Detected', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return hexagon_centers

In [ ]:
detect_hexagons(X[12])